In [3]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from load_dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [6]:
extract_data=load_pdf_file("/Users/jaganpradhan/Desktop/Level3/MediRag/data")


In [7]:

print(len(extract_data))

637


In [8]:
from typing import List
from langchain_core.documents import Document

def fiter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    minimal_docs = []
    for doc in docs:
        minimal_doc = Document(page_content=doc.page_content, metadata={})
        minimal_docs.append(minimal_doc)
    return minimal_docs

In [9]:
#Split the dcocuments into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
def split_documents(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
    )
    text_chunks = text_splitter.split_documents(minimal_docs)
    return text_chunks

In [10]:
minimal_docs=fiter_to_minimal_docs(extract_data)
text_chunks=split_documents(minimal_docs)
print(len(text_chunks))

5961


In [11]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

embed_query=embeddings.embed_query("Hello world")
print(len(embed_query))

384


In [22]:
load_dotenv()
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
HUGGINGFACE_API_KEY=os.getenv("HUGGINGFACE_API_KEY")

In [23]:
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["HUGGINGFACE_API_KEY"]=HUGGINGFACE_API_KEY

In [24]:
from pinecone import Pinecone
import os
from dotenv import load_dotenv

load_dotenv()

pinecone_api_key = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
print("Pinecone Connected Successfully!")

Pinecone Connected Successfully!


In [26]:
from pinecone import ServerlessSpec
index_name = "rag-project-medical-asisstant"
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384, #size of the embedding vector
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    print(f"Index '{index_name}' created successfully.")
index = pc.Index(index_name)

Index 'rag-project-medical-asisstant' created successfully.


In [27]:
from langchain_pinecone import PineconeVectorStore
docssearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

In [28]:
#Load from existing index
docssearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [29]:
#Add more documents to existing data
docnew=Document(
    page_content="Hello My name is Jagan Pradhan",
    meta_data={"source":"Mac Os"}
)

In [30]:
docssearch.add_documents(documents=[docnew])

['44ad1708-f82c-4341-b90b-d0387aa7711b']

In [31]:
retriever=docssearch.as_retriever(search_type="similarity" ,search_kwargs={"k":3})

In [32]:
retrieved_docs=retriever.invoke("What is Acne")

In [35]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceTB/SmolLM3-3B",  
    task="text-generation"
)

# Wrap with chat interface (handles templating automatically)
chat = ChatHuggingFace(llm=llm)

In [36]:
response = chat.invoke(
    "Explain the causes and treatment options for acne.")
print(response.content)

<think>
Okay, so I need to explain the causes and treatment options for acne. Let me start by recalling what I know about acne. Acne is a common skin condition that involves pimples, blackheads, whiteheads, and sometimes cysts. I remember that it's related to the oil glands in the skin, but I'm not exactly sure about the details.

First, the causes. I think it's linked to hormones, especially testosterone, which is why teenagers get acne. But wait, isn't it also influenced by genetics? Maybe if your parents had acne, you're more likely to too. Then there's the role of bacteria on the skin, like Propionibacterium acnes (P. acnes). These bacteria can grow in the pores when there's too much oil, leading to inflammation. Oh, and maybe stress? I've heard that stress can trigger acne breakouts. Also, certain medications like corticosteroids or lithium might contribute. Maybe even diet? I've read that a diet high in sugar or dairy can be a factor, though I'm not sure if the evidence is strong

In [37]:
from langchain_openai import ChatOpenAI  # Switch to OpenAI wrapper for OpenRouter compatibility
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import os

# Load .env
load_dotenv()

# Get your OpenRouter API key
openrouter_api_key = os.getenv("OPENROUTER_API_KEY") # VARIABLE name remain same for all model of openrouter_api

if not openrouter_api_key:
    raise ValueError("OpenRouter API key not found! Check your .env file.")

# Initialize ChatOpenAI pointing to OpenRouter (for Anthropic model)
chat = ChatOpenAI(
    model="nvidia/nemotron-nano-12b-v2-vl:free", #change the model name here for different model of openrouter
    api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1", # OpenRouter base URL same as before
)

In [38]:
from langchain_core.prompts import PromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate,ChatPromptTemplate

prompt_template = """You are a helpful medical assistant. Use the following context to answer the question at the end.
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
Context: {context}
Question: {question}
Helpful Answer:"""
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

In [39]:
from langchain_core.runnables import RunnableSequence, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# Step 1: Convert docs + question to prompt input
def docs_to_prompt_input(docs, question):
    return {"context": "\n".join([d.page_content for d in docs]), "question": question}

docs_to_prompt_runnable = RunnableLambda(lambda inputs: docs_to_prompt_input(inputs["docs"], inputs["question"]))

# Step 2: Wrap prompt -> chat as a Runnable
question_answer_chain = RunnableLambda(lambda inputs: chat.invoke(prompt.format(**inputs)))

# Step 3: Full RAG chain
rag_chain = RunnableSequence(
    # Retriever now invoked with query, returns docs list
    RunnableLambda(lambda question: {"docs": retriever.invoke(question), "question": question}),
    docs_to_prompt_runnable,
    question_answer_chain,
    parser
)

# Step 4: Invoke chain dynamically
question = "What is Acne"
response = rag_chain.invoke(question)
print(response)
 

Acne is a common skin condition caused by clogged pores due to excess oil (sebum), dead skin cells, and bacteria. It typically appears as pimples (blackheads, whiteheads, or inflamed zits) on the face, chest, and back. The medical term *acne vulgaris* affects nearly 17 million people in the U.S. Key points:  
- **Cause**: Blocked hair follicles or pores lead to sebum accumulation, bacterial growth, and inflammation.  
- **Severity**: Ranges from mild (comedo-type acne) to severe (nodules, cysts).  
- **Treatment**: Options include topical medications (e.g., comedolytic drugs), oral antibiotics, or prescription drugs like isotretinoin (Accutane) for severe cases.  

If you suspect acne, consult a healthcare provider for tailored advice.

